In [1]:
import urllib3
urllib3.disable_warnings()
import json
import datetime
import csv
import time
import pandas as pd

* El ejemplo sale de: https://github.com/minimaxir/facebook-page-post-scraper/blob/master/examples/how_to_build_facebook_scraper.ipynb

* Usa urllib3 en vez de urllib2

* La adaptación de "testFacebookPageData" sale de:  
https://urllib3.readthedocs.io/en/latest/user-guide.html

In [3]:
# El app_id salió de registrarme como desarrollador en facebook, y creando una nueva aplicaicón.
# luego yendo a dashboard obtenes el app_idd, y app_secret
# https://developers.facebook.com/apps/133254600775938/dashboard/

app_id = "133254600775938"

app_secret = "c24db12ccb6a583d8e212600c1bed43e" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#page_id = 'nytimes'
#page_id = 'www.salesintacc.com.ar'
#page_id = 'upsoclsabores'
page_id = 'cucinareok'

In [4]:
def request_until_succeed(url):
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    success = False
    while success is False:
        try:
            response = http.urlopen('GET',url)
            if response.status == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return req

In [5]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.11" #cambié v2.4 a v2.11, funcionaba igual con v2.4
    node = "/" + page_id + "/feed"
    parameters = "/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # Esto está cambiado respecto al ejemplo
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    data = json.loads(request_until_succeed(url).data.decode('latin-1'))
    return data
    
#test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
#print(json.dumps(test_status, indent=4, sort_keys=True))

In [6]:
#Estoy levantando 1000 posts
salida_all = getFacebookPageFeedData(page_id, access_token, 1)

columns = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']

df_posts = pd.DataFrame(columns=columns)

MaxRetryError: HTTPSConnectionPool(host='graph.facebook.com', port=443): Max retries exceeded with url: /v2.11/cucinareok/feed/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=1&access_token=133254600775938|c24db12ccb6a583d8e212600c1bed43e (Caused by ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)))

In [168]:
def scrapPosts(post):
    post_from = '' if 'from' not in post.keys() else post['from']['id']
    post_id = post['id']
    post_name = '' if 'name' not in post.keys() else post['name']
    post_type = '' if 'type' not in post.keys() else post['type']
    post_message = '' if 'message' not in post.keys() else post['message']
    post_link = '' if 'link' not in post.keys() else post['link']
    post_shares = '' if 'shares' not in post.keys() else post['shares']['count']
    created_time = datetime.datetime.strptime(post['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    created_time = created_time + datetime.timedelta(hours=-5) # EST
    created_time = created_time.strftime('%Y-%m-%d %H:%M:%S')    

    return(post_from, post_id, post_name, post_type, post_message, post_link, post_shares, created_time) 
    

In [169]:
n_posts = len(salida_all['data'])

for i in range(n_posts):
    df = pd.DataFrame(list(scrapPosts(salida_all["data"][i])),index = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']).transpose()
    df_posts = df_posts.append(df, ignore_index=True)    


In [170]:
df_posts

,post_from,post_id,post_name,post_type,post_message,post_link,post_shares,created_time
0,262755230730537,262755230730537_543142066025184,Empanadas de mariscos,video,¿Empanadas de #carne? ¿De jamón y queso? \n\nE...,https://www.facebook.com/cucinareok/videos/543...,1,2017-11-15 23:00:00
1,262755230730537,262755230730537_543096902696367,"Cucinare y Gato Dumas Colegio de Gastronomía, ...",link,Renovamos el compromiso con los fanáticos de n...,https://www.cucinare.tv/2017/11/14/cucinare-y-...,,2017-11-15 20:32:00
2,262755230730537,262755230730537_543146876024703,Rolls de pizza,video,"¡Decile ""chau"" a la clásica pizza 🍕!\n\nEste r...",https://www.facebook.com/cucinareok/videos/543...,451,2017-11-15 19:00:00
3,262755230730537,262755230730537_543053046034086,,photo,"En #Cucinare nos estamos preparando con ""tutti...",https://www.facebook.com/cucinareok/photos/a.2...,9,2017-11-15 16:31:13
4,262755230730537,262755230730537_543121609360563,Frozen de banana y arándanos,video,¿Tenés arándanos y banana? \n\nAprovechá el ca...,https://www.facebook.com/cucinareok/videos/543...,656,2017-11-15 14:50:32
5,262755230730537,262755230730537_543041352701922,Pizza de espinaca con salsa blanca,video,¿Una clásica de muzza? ¡NOOO! \n\nAnimate a in...,https://www.facebook.com/cucinareok/videos/543...,1100,2017-11-15 10:56:08
6,262755230730537,262755230730537_542956589377065,Cucinare: Red velvet cookies,video,Arrancar el día con unos buenos mates y unas d...,https://www.facebook.com/cucinareok/videos/542...,221,2017-11-15 07:36:33
7,262755230730537,262755230730537_542666492739408,"Lasagna de espinaca, ricotta y queso",video,"Como todos los martes, las #pastas llegan a la...",https://www.facebook.com/cucinareok/videos/542...,1316,2017-11-14 23:00:00
8,262755230730537,262755230730537_542666352739422,Fideos con el sello de Dolce & Gabbana: ¿la pa...,link,No vas a creer el precio de un paquete de spag...,https://www.cucinare.tv/2017/11/14/fideos-con-...,2,2017-11-14 20:31:00
9,262755230730537,262755230730537_542654082740649,Torta salada de canastitas,video,"Si te gustan las canastitas, ¡mucho más te va ...",https://www.facebook.com/cucinareok/videos/542...,678,2017-11-14 20:00:00


In [105]:
salida_individual = procesaPosts(salida)

for i in range(ncomm):

    df = pd.DataFrame(list(scrapComments(i)), index=['comm_id','comm_msg','comm_date']).transpose()

    df_comms = df_comms.append(df, ignore_index=True)



NameError: name 'ncomm' is not defined

In [ ]:
dict_keys(['link', 'created_time', 'shares', 'from', 'type', 'comments', 'message', 'id', 'name'])

In [15]:
salida_all = getFacebookPageFeedData(page_id, access_token, 10)

salida1 = salida_all["data"][0]

salida2 = salida_all["data"][1]

salida3 = salida_all["data"][2]

salida4 = salida_all["data"][3]

salida5 = salida_all["data"][4]

salida6 = salida_all["data"][5]

salida7 = salida_all["data"][6]

salida8 = salida_all["data"][7]

salida9 = salida_all["data"][8]

salida10 = salida_all["data"][9]

In [18]:
salida10.keys()

dict_keys(['link', 'created_time', 'shares', 'from', 'type', 'comments', 'message', 'id', 'name'])

In [40]:
salida1['message']

'¡Decile "chau" a la clásica pizza 🍕!\n\nEste roll es increíblemente rico y mucho más fácil 😍😍😍 #inlove #Cucinare #RecetasParaCompartir'

In [23]:
salida10['id']

'262755230730537_542653566074034'

In [24]:
#salida10['comments']['data']

In [10]:
salida_comments = salida['comments']['data']

In [97]:
def scrapComments(pos):

    comm_id = salida_comments[pos]['id']
    comm_msg = salida_comments[pos]['message']
    comm_date = datetime.datetime.strptime(salida_comments[pos]['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    comm_date = comm_date + datetime.timedelta(hours=-5) # EST
    comm_date = comm_date.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    return(comm_id, comm_msg, comm_date)    

In [98]:
#import pandas as pd

df_comms = pd.DataFrame(columns=['comm_id','comm_msg','comm_date'])

In [99]:
#import pandas as pd
#list(scrapComments(1))
#pd.DataFrame(scrapComments(1))

ncomm = len(salida_comments)

for i in range(ncomm):

    df = pd.DataFrame(list(scrapComments(i)), index=['comm_id','comm_msg','comm_date']).transpose()

    df_comms = df_comms.append(df, ignore_index=True)


In [2]:
import os

os.getcwd()

'C:\\Users\\axmarr\\Documents\\Python\\WinPython-64bit-3.6.1.0Qt5\\notebooks'

In [3]:
os.chdir('C:\\Users\\axmarr\\Documents\\Varios\\Maestria\\NLP\\')

In [4]:
import pandas as pd

In [7]:
comentarios = pd.read_csv('df_comms.csv', encoding='latin-1')

In [24]:
text = u'This dog \U0001f602'
print(text) # with emoji

This dog 😂


b'This dog \xf0\x9f\x98\x82'

In [8]:
#Desde acá: https://nlp.stanford.edu/software/CRF-NER.shtml#Download
#Se baja: https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
#Se descomprime el .zip y a la carpeta que está adentro se le cambia el nombre de: stanford-ner-2017-06-09 a: stanford-ner 

#import os
java_path = "C:/Program Files/Java/jre1.8.0_131/bin/java.exe"
os.environ['JAVAHOME'] = java_path

#os.getcwd()
#Hay que cambiar el directorio y que apunte donde está la carpeta que descomprimimos
#os.chdir("C:/Users/axmarr/Documents/Varios/maestria/NLP/")

# -*- coding: utf-8 -*-

#from nltk.tag import StanfordNERTagger
#from nltk.tokenize import word_tokenize

st = StanfordNERTagger('stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
					   'stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

#st = StanfordNERTagger('../../stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
#					   '../../stanford-ner/stanford-ner.jar',
#					   encoding='utf-8')




C:\Users\axmarr\Documents\Python\WinPython-64bit-3.6.1.0Qt5\python-3.6.1.amd64\lib\site-packages\nltk\tag\stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)


In [15]:
#import nltk

#nltk.download('punkt')

text = 'While in France, Christine lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)

type(classified_text)

list

In [76]:
import pandas as pd

#list comprehension
#my_list = [pers for pers in classified_text if 'PERSON' in pers]
#my_list2 = [pers for pers in classified_text if pers[1] =='PERSON']
#lambda
#my_list3 = filter(lambda pers: pers[1] =='PERSON', classified_text)

comments = pd.read_csv('df_comms.csv',encoding='latin-1')

comm_str = comments.comm_msg.to_json()

comm_token = word_tokenize(comm_str)

comm_class = st.tag(comm_token)

comm_personas = [pers for pers in comm_class if pers[1] =='PERSON']


In [78]:
#comm_personas

In [80]:
text = u'This dog \U0001f602 is in Buenos Aires'

text_token = word_tokenize(text)

text_class = st.tag(text_token)

text_class

#Juntar entidades como chunks y no separadas:
#https://stackoverflow.com/questions/30664677/extract-list-of-persons-and-organizations-using-stanford-ner-tagger-in-nltk


[('This', 'O'),
 ('dog', 'O'),
 ('😂', 'O'),
 ('is', 'O'),
 ('in', 'O'),
 ('Buenos', 'LOCATION'),
 ('Aires', 'LOCATION')]

In [89]:
#Filtrar emojis del string o solo quedarse con los emojis
import re

text = 'Juanma😂🤣'
print(text) # with emoji

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

print(emoji_pattern.sub(r'', text)) # no emoji

emoji_pattern.findall(text) #solo emojis

Juanma😂🤣
Juanma🤣


['😂']

In [87]:
emoji_pattern.findall(text)

['🌷💖']